# DS 2010 Final Project - Data Cleaning

## Import Packages

In [1]:
import pandas as pd
import math
import numpy as np
import datetime as dt

## Import Data

In [2]:
## Predictors ##
trips_data = pd.read_csv("Raw_Data/Trips_by_Distance.csv")
covid_19_data = pd.read_csv("Raw_Data/all-states-history-correct-range.csv")

## Response ##
# Starts Apr 23, Ends Nov 9
anxiety_depression_data = pd.read_csv("Raw_Data/Indicators_of_Anxiety_or_Depression_Based_on_Reported_Frequency_of_Symptoms_During_Last_7_Days.csv")

## Clean trip data

In [3]:
# Reduce dimensionality from county -> state 
td = trips_data.groupby(by = ['Date', 'State Postal Code']).sum().drop(columns= ['State FIPS', 'County FIPS']).reset_index()
td['Period'] = np.floor(td.index / 51 / 7)
# Create Period column
td2 = td.groupby(by = ['Period', 'State Postal Code']).mean().reset_index()
td2["Period"] = td2["Period"] - 1
td2.loc[td2["Period"] == -1,"Period"] = 0
# Index 1-11 are 1 week
# Index 12-16 are break
for i in range(12, 17):
    td2.loc[td2["Period"] == i,"Period"] = 12

# Index 17|18, 19|20, 21|22, 23|24, 25|26, 27|28 are one time period
# Thus this code makes the two week time fromes to one period
period = 13
for i in range(0, 12, 2):
    week = i + 17
    td2.loc[td2["Period"] == week,"Period"] = period
    td2.loc[td2["Period"] == week + 1,"Period"] = period
    period += 1

# Sum the period and state to get the trips per time period
td2 = td2.groupby(by= ["Period", "State Postal Code"]).mean().reset_index()

# Remove 12th time period because it is the break
td2 = td2[td2['Period'] != 12.0]

# Increment all the time periods before 12 by 1
td2['Period'] = td2.apply(lambda x:
                        x['Period']+1
                        if x['Period'] < 12 
                        else
                        x['Period'], axis=1)

trip_df = td2.copy()

# Rename State column to align with other data
trip_df['State'] = trip_df['State Postal Code']
trip_df = trip_df.drop('State Postal Code', axis=1)

In [4]:
trip_df.to_csv("Clean_Data/clean_trip_data.csv", index= False)

## Clean COVID-19 Data

### Prepare covid data

In [5]:
# Only take desired columns from the data
valuable_cols = ['state', 'date', 'death', 'deathConfirmed', 
                 'hospitalizedCurrently', 'positiveCasesViral', 
                 'positiveIncrease', 'totalTestsPeopleViral', 
                 'totalTestsPeopleViralIncrease', 'totalTestsViral', 'positiveTestsViral']
covid_data = covid_19_data.copy()[valuable_cols]

# Convert string to datetime
covid_data['date'] = covid_data['date'].astype('datetime64[ns]')

# 50 States + 1 Federal District (DC: District of Columbia)
remove_states = ['AS', 'PR', 'GU', 'MP', 'VI']
# AS: American Samoa
# PR: Puerto Rico
# GU: Guam
# MP: Northern Mariana Islands
# VI: US Virgin Islands
states = list(set(covid_data['state'].values))
final_states = [item for item in states if item not in remove_states]

### Create Time Period column

In [6]:
# These ranges match the time periods within our response data.
ranges_list = ['4/23/2020-5/5/2020', '5/7/2020-5/12/2020', '5/14/2020-5/19/2020', '5/21/2020-5/26/2020',
               '5/28/2020-6/2/2020', '6/4/2020-6/9/2020', '6/11/2020-6/16/2020', '6/18/2020-6/23/2020',
               '6/25/2020-6/30/2020', '7/2/2020-7/7/2020', '7/9/2020-7/14/2020', '7/16/2020-7/21/2020',                  
               '8/19/2020-8/31/2020', '9/2/2020-9/14/2020', '9/16/2020-9/28/2020', '9/30/2020-10/12/2020', 
               '10/14/2020-10/26/2020', '10/28/2020-11/9/2020']

# Convert ranges_list to a list of lists of start and end datetimes
period_list = []
for r in ranges_list: # '4/23/2020-5/5/2020'
    r_list = []
    for date_str in r.split('-'): # [4/23/2020, 5/5/2020]
        date = dt.datetime.strptime(date_str, '%m/%d/%Y').date() # 4/23/2020 -> datetime
        r_list.append(date) # [start,end]
    period_list.append(r_list) # [[start,end],[start,end]..]

# Create Time Period column
time_periods = [] 
    # 1-n: respective period by number 
    # -1: occur before first period
    # -2: occur after last period
    # 0: within period but not included
for index, row in covid_data.iterrows():
    true_period = np.NaN
    if row['date'] < period_list[0][0]:
        true_period = -1 # if occur before first period
    elif row['date'] > period_list[-1][-1]:
        true_period = -2 # if occur after last period
    else: # else occur within a period
        for period in period_list:
            if period[0] <= row['date'] <= period[1]:
                true_period = period_list.index(period) + 1
    time_periods.append(true_period)
# Add Time Period column to covid_data
covid_data['Period'] = time_periods

# Make a copy before cleaning
data = covid_data.copy()

# Remove rows with invalid time periods
data.drop(data.loc[data['Period'] == -1].index, inplace=True)
data.drop(data.loc[data['Period'] == -2].index, inplace=True)
data.drop(data.loc[data['Period'] == np.NaN].index, inplace=True)
data = data.reset_index(drop=True)

### Fill NaN and groupby Time Period for each state

In [7]:
# empty list to fill with each state's cleaned data
clean_data = []

# set death equal to max from death and deathConfirmed
data["death"] = data[["death", "deathConfirmed"]].max(axis=1)
# set positiveCasesViral equal to max from positiveCasesViral,totalTestsViral, and totalTestsPeopleViral
data["positiveCasesViral"] = data[["positiveCasesViral", "totalTestsViral", "totalTestsPeopleViral"]].max(axis=1)
# drop redundant columns
clean_df = data.drop(columns=["deathConfirmed", "totalTestsViral", "totalTestsPeopleViral"], axis=1)

for state in final_states:
    state_data = clean_df.loc[clean_df['state'] == state].reset_index(drop=True)
    # Convert cumulative columns -> increase-by columns
    # by calculating difference of the day prior from each day
    state_data['deathIncrease'] = state_data['death'].diff(+1)
    state_data['positiveTestsViralIncrease'] = state_data['positiveTestsViral'].diff(+1)
    state_data['positiveCasesIncrease'] = state_data['positiveCasesViral'].diff(+1)
    # Positivity Rate interaction term
    state_data["positivityRate"] = state_data["positiveTestsViralIncrease"] / state_data["totalTestsPeopleViralIncrease"]
    # drop redundant columns
    state_data = state_data.drop(columns=['totalTestsPeopleViralIncrease'], axis=1)
    # replace inf with 1
    state_data = state_data.replace(math.inf, 1)
    state_data = state_data.replace(-math.inf, 1)
    # replace null positivity rates with 0
    state_data["positivityRate"] = state_data["positivityRate"].replace(np.NaN, 0)
    
    # reduce dimensionality from days to periods
    state_data = state_data.groupby(['Period', 'state']).mean().reset_index()
    clean_data.append(state_data)

# combine states to final df
clean_df = pd.concat(clean_data)
# drop rows with null values
ignore_columns = ['Period', 'state','positiveTestsViral', 'positiveTestsViralIncrease']
drop_by_columns = [item for item in list(clean_df.columns) if item not in ignore_columns]
clean_df = clean_df.dropna(subset=drop_by_columns)
covid_df = clean_df.copy()
covid_df['State'] = covid_df['state']
covid_df = covid_df.drop('state', axis=1)

In [8]:
covid_df.to_csv("Clean_Data/clean_covid_data.csv", index= False)

## Clean Anxiety/Depression Data

In [9]:
data = anxiety_depression_data.copy()

In [10]:
# Remove demographic rows and uneeded columns
clean_data = data.loc[data['State'] != 'United States']
clean_data = clean_data.drop(['Phase', 'Group', 'Subgroup', 'Time Period Label',
                             'Low CI', 'High CI', 'Confidence Interval', 'Quartile range'], axis=1)

# Break each target into a unique dataframe
depression_data = clean_data.loc[clean_data['Indicator'] == 'Symptoms of Depressive Disorder']
anxiety_data = clean_data.loc[clean_data['Indicator'] == 'Symptoms of Anxiety Disorder']
both_data = clean_data.loc[clean_data['Indicator'] == 'Symptoms of Anxiety Disorder or Depressive Disorder']

# Merge each target back into one dataframe on State and Time Period
# Each target now is displayed in a column, and the height of the column is divided by 3
merged_data = pd.merge(depression_data, anxiety_data, on=['State', 'Time Period'])
merged_data = pd.merge(merged_data, both_data, on=['State', 'Time Period'])

# Clean final dataframe
merged_data = merged_data.drop(['Indicator_x', 'Indicator_y', 'Indicator'], axis=1)
merged_data.columns = ['State', 'Period', 'Depression_Score', 'Anxiety_Score', 'Mix_Score']

In [11]:
# Dictionary of states, used to change name -> acronym
states_hash = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Federated States Of Micronesia': 'FM',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Marshall Islands': 'MH',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': ' ',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands': 'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Palau': 'PW',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'}

In [12]:
# Change state name into acronym
merged_data['State'] = merged_data.apply(lambda x: states_hash[x['State']], axis=1)

In [13]:
response_df = merged_data.copy()

In [14]:
response_df.to_csv("Clean_Data/clean_label_data.csv", index= False)

# Merge predictor and response datasets

In [15]:
# final_df = response_df + covid_df + trip_df
final_df = pd.merge(response_df, covid_df, on=['Period', 'State'])
final_df = pd.merge(final_df, trip_df, on=['Period', 'State'])

In [16]:
# desired column order 'positivityRate', 'positiveCasesIncrease', 'positiveTestsViralIncrease', 
columns = ['State', 'Period', 'Mix_Score', 'Depression_Score', 'Anxiety_Score',
           'deathIncrease', 'death', 'hospitalizedCurrently', 
           'positivityRate',
           'positiveTestsViralIncrease', 'positiveTestsViral',
           'positiveCasesIncrease', 'positiveCasesViral',
           'Population Staying at Home', 'Population Not Staying at Home',
           'Number of Trips', 'Number of Trips <1', 'Number of Trips 1-3',
           'Number of Trips 3-5', 'Number of Trips 5-10', 'Number of Trips 10-25',
           'Number of Trips 25-50', 'Number of Trips 50-100',
           'Number of Trips 100-250', 'Number of Trips 250-500',
           'Number of Trips >=500']
final_df = final_df[columns]

In [17]:
statePop = pd.read_csv("StatePop.csv") # state populations
# turn state name into acronym
statePop['State'] = statePop.apply(lambda x: states_hash[x['State']], axis=1)
# add population column to final_df
final_df = pd.merge(final_df, statePop, how='inner', on=['State'])

In [18]:
# create list of feature column names 'positivityRate',
features = list(final_df.columns)
for col in ['State', 'Period', 'Depression_Score', 'Anxiety_Score', 'Mix_Score', ]:
    features.remove(col)

In [19]:
# normalize features by population, except positivtyRate
normalized_df = final_df.copy()
for feature in features:
    normalized_df.loc[:,feature] = normalized_df.loc[:,feature]/final_df['Pop']

In [20]:
# drop redundant columns
normalized_df = normalized_df.drop(columns=["Pop", "density"], axis=1)

In [21]:
normalized_df.to_csv("Clean_Data/final_data.csv", index= False)